# Automation for IB portfolios

Latest version: 2024-08-27  
Author: MvS

## Description

Notebook to test `ib_async` module to interact with IB portfolios

## Result


### Warning: This notebook will place live orders


In [ ]:
from ib_async import *
import yfinance as yf

import pandas as pd
import numpy as np
from dotenv import dotenv_values

import datetime
import time
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO, format="%(asctime)s %(message)s")

# log everything
if True:
    util.logToConsole(logging.INFO)

# ib_sysnc: notebook relevant
util.startLoop()

# read env vars
env_dict = dotenv_values("../.env")

ib = IB()

if ~ib.isConnected():
    ib.connect(
        env_dict['IB_API_IP'],
        int(env_dict['IB_API_PORT']),
        clientId=int(env_dict['IB_API_KEY']),
    )

### Get liquidation value of whole account

In [ ]:
liq_balance = [
    (v.account, v.value, v.currency)
    for v in ib.accountValues()
    if v.tag == 'NetLiquidationByCurrency' and v.currency == 'BASE'
]

logging.info('Printing liquidation value:')
for counter, (acc, bal, curr) in enumerate(liq_balance, 1):
    logging.info(f"{acc}, {counter:0d}: {float(bal):12.2f} {curr}")

### Unpack cash and stock balance

In [ ]:
cash_balance = [
    (v.account, v.value, v.currency)
    for v in ib.accountValues()
    if v.tag == 'CashBalance' and v.currency != 'BASE'
]
stock_balance = [
    (v.account, v.value, v.currency)
    for v in ib.accountValues()
    if v.tag == 'StockMarketValue' and v.currency != 'BASE'
]

logging.info('Printing cash balance:')
for counter, (acc, bal, curr) in enumerate(cash_balance, 1):
    logging.info(f"{acc}, {counter:0d}: {float(bal):12.2f} {curr}")

logging.info('Printing stock balance:')
for counter, (acc, bal, curr) in enumerate(stock_balance, 1):
    logging.info(f"{acc}, {counter:0d}: {float(bal):12.2f} {curr}")

# ib.accountValues()

In [ ]:
amd = Stock('AMD', 'SMART', 'EUR')

assert len(ib.reqContractDetails(amd)) == 1
amd

In [ ]:
ib.qualifyContracts(amd)
amd

In [ ]:
contract_32596680 = Contract(conId=32596680)

ib.qualifyContracts(contract_32596680)

assert contract_32596680 == amd

In [ ]:
order = LimitOrder(action='BUY', totalQuantity=100, lmtPrice=131.22)

In [ ]:
limitTrade = ib.placeOrder(amd, order)

ib.sleep(1)
assert limitTrade.orderStatus.status == 'Submitted'

In [ ]:
limitTrade.orderStatus.status

find contract in postions

In [ ]:
for pos in ib.positions():
    if pos.contract.symbol == 'AMD':
        logging.info('found')
        cur_contract = Contract(conId=pos.contract.conId)

use substitution by id

In [ ]:
assert contract_32596680 == amd
logging.info(f"Before: {cur_contract}")
ib.qualifyContracts(cur_contract)
logging.info(f"Before: {cur_contract}")
cur_contract

submit stop loss order

In [ ]:
stopLossLimitOrder = StopLimitOrder(
    action='SELL',
    totalQuantity=100.0,
    lmtPrice=118.22,
    stopPrice=120.22,
)
stopLossLimitOrder.outsideRth = True
stopLossLimitOrder.tif = 'GTC'

limitTrade = ib.placeOrder(cur_contract, stopLossLimitOrder)
ib.sleep(1)
assert limitTrade.orderStatus.status == 'PreSubmitted'

find stop-loss in open trades

In [ ]:
for order in ib.reqAllOpenOrders():
    if order.contract.symbol == 'AMD' and order.order.orderType == 'STP LMT':
        logging.info('found')
        break

order

In [ ]:
cancelled_order = ib.cancelOrder(order.order)
ib.sleep(1)
cancelled_order

In [ ]:
logging.info(limitTrade.orderStatus)

In [ ]:
ib.disconnect()